# This NB was created to generate primers for KOing genes in yeast by replacing the gene with KanMX


In [ ]:
### the KanMX CDS is identical for all KOs, an upstream fragement and downstream fragment are created specific 
### for each gene

### the upstram and dwostream fragement have approx. 400bp homologous to site to delete and 40bp homologous to the 
#KanMX cassette



In [1]:
import  Bio
from Bio.Seq import Seq
import DNA_tools as dt
import os
from Bio import SeqIO
import importlib #importlib.reload(ca)
import pandas as pd
import numpy as np

In [135]:
path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/Betaxanthin/DNA_parts/KO_confirmations/'
os.listdir(path2parts)

['vps34.gb',
 'cox12.gb',
 'shy1.gb',
 'rcr2.gb',
 'cax4.gb',
 '.DS_Store',
 'nam7.gb',
 'dld2.gb',
 'yor1.gb',
 'kanmx.gb',
 'kgd2.gb',
 'mip1.gb',
 'yer084w.gb',
 'yku70.gb',
 'cur1.gb',
 'rim13.gb',
 'sgm1.gb']

In [41]:
#some parameters:
homology_length= 400 #length of the homology region upstream and downstream of a CDS
out_gap= 170 #length outside of integration sequences used as gap for confirmation primers

In [136]:
## dict storing file names of parts:
genes2KO=['cax4', 'cox12','cur1','dld2','kgd2', 'mip1','nam7','rcr2','sgm1', 'shy1', 'vps34', 'yku70', 'yor1','yer084w','rim13']
genes2KO= [g.upper() for g in genes2KO]
part_file={}
for gene in genes2KO:
    part_file[gene]= gene+".gb"

part_seq={}
part_seq['KanMX']='taggtctagagatctgtttagcttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgagctctcgagaacccttaat'

#Defines the Overhangs with KanMX  
KanMX_overhang_Up = dt.reverse_complement(part_seq['KanMX'][0:40]) #we copy the reverse complement for the upstream up
KanMX_overhang_Dw = part_seq['KanMX'][-40:]


In [58]:
-homology_length-out_gap

-570

In [137]:
#extract the sequence [upper homology:CDS:lower homology]

for gene in genes2KO:

    #open .gb file
    start=0
    stop=0
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    for rec in recs:
        #loop through feature to features to find the location  of the CDS:
        for feat in rec.features:
            if gene + ' gene'in feat.qualifiers['label'] or gene in feat.qualifiers['label']:
                location= feat.location
                start = location.start.position
                stop = location.end.position

        #stores seq in dict
        part_seq[gene] = str(rec.seq[start-homology_length-out_gap:stop+homology_length+out_gap])
        print(gene, start-homology_length-out_gap,stop+homology_length+out_gap)


CAX4 308 2168
COX12 106 1498
CUR1 30 1929
DLD2 193 2926
KGD2 9 2541
MIP1 88 4993
NAM7 179 4235
RCR2 214 1987
SGM1 19 3283
SHY1 285 2595
VPS34 36 3804
YKU70 60 3009
YOR1 528 6102
YER084W 246 1773
RIM13 93 3417


In [138]:
#design the KanMX primers
Kan_up= dt.make_anneal(part_seq['KanMX'].upper(), 'F' , 50)
Kan_dw= dt.make_anneal(part_seq['KanMX'], 'R' , 50)

In [139]:
importlib.reload(dt)
UF_list=[]
UR_list=[]
DF_list=[]
DR_list=[]
for gene in genes2KO:
    #design the upstream F and R primers
    seq= part_seq[gene]
    UF = dt.make_anneal(seq, 'F' , 50)
    UR = dt.make_anneal(seq[0:homology_length], 'R' , 50)
    #design the downstream F and R primers:
    DF = dt.make_anneal(seq[-homology_length:], 'F' , 50)
    DR = dt.make_anneal(seq, 'R' , 50)

    #copy the KanMX 30bp overhang to the UR and DF:
    UR =  (( KanMX_overhang_Up).lower() + UR[0], UR[1])

    DF =  ((KanMX_overhang_Dw).lower() + DF[0], DF[1])
    
    UF_list.append(UF)
    UR_list.append(UR)
    DF_list.append(DF)
    DR_list.append(DR)

In [147]:
[1,2,3,4,5][-3:]

[3, 4, 5]

In [149]:
importlib.reload(dt)
UF_list=[]
UR_list=[]
DF_list=[]
DR_list=[]
for gene in genes2KO:

    #design the upstream F and R primers
    seq= part_seq[gene]
    UF = dt.make_anneal(seq[out_gap:out_gap+homology_length], 'F' , 50)
    #design primers from leaving an upstream outgap and homology_length bp to CDS
    UR = dt.make_anneal(seq[out_gap:out_gap+homology_length], 'R' , 50)
    
    #design the downstream F and R primers:
    DF = dt.make_anneal(seq[-homology_length-out_gap:-out_gap], 'F' , 50)
    DR = dt.make_anneal(seq[-homology_length-out_gap:-out_gap], 'R' , 50)

    #copy the KanMX 30bp overhang to the UR and DF:
    UR =  (( KanMX_overhang_Up).lower() + UR[0], UR[1])

    DF =  ((KanMX_overhang_Dw).lower() + DF[0], DF[1])
    
    UF_list.append(UF)
    UR_list.append(UR)
    DF_list.append(DF)
    DR_list.append(DR)

In [150]:
#sort the ouptut into a nice data frame

all_primers =[]
all_temp =[]
position = []
Gene= []
NA =[]
Primer_name=[]
primer_n = 0

for i, gene in enumerate(genes2KO):

        
    all_primers.append(UF_list[i][0])
    all_temp.append(str(round(UF_list[i][1] ,1) +6 ) + ' phus')      #we add 6C for NEB phusion
    position.append('UF')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('BTX.'+str(i+1+3*i +primer_n ).zfill(3))

    
    all_primers.append(UR_list[i][0])
    all_temp.append(str(round(UR_list[i][1],1) + 6)+ ' phus')   
    position.append('UR')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('BTX.'+str(i+2+3*i +primer_n ).zfill(3))
    
    all_primers.append(DF_list[i][0])
    all_temp.append(str(round(DF_list[i][1],1) + 6)+ ' phus')   
    position.append('DF')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('BTX.'+str(i+3+3*i +primer_n ).zfill(3))
    
    all_primers.append(DR_list[i][0])
    all_temp.append(str(round(DR_list[i][1],1) + 6)+ ' phus')   
    position.append('DR')               
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('BTX.'+str(i+4+3*i +primer_n ).zfill(3))
 
#add the KanMX primers
all_primers.append(Kan_up[0])
all_temp.append(str(round(Kan_up[1],1) + 6)+ ' phus')   
position.append('F')
Gene.append('KanMX (pCfB 2312)')
NA.append('na')
Primer_name.append('BTX.'+'XXX')

all_primers.append(Kan_dw[0])
all_temp.append(str(round(Kan_dw[1],1) + 6)+ ' phus')   
position.append('R')
Gene.append('KanMX (pCfB 2312)')
NA.append('na')
Primer_name.append('BTX.'+'XXX')


#some primers are over IDT 60bp limit we shorten there homology region to KanMX:
for i, primer in enumerate(all_primers):
    if len(primer) >60:
        all_primers[i] = primer[-60:]
        if  (sum([bp.islower() for bp in all_primers[i]]) < 30):
            print('WARNING your homology overhang was cut to <30bp')                    
                            
        


In [151]:
df = np.stack([Primer_name, all_primers,position, all_temp,NA, Gene, NA, NA] ).T
Primer_Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Primer_Table.set_index('Primer Name')

Primer_Table.to_csv('/Users/paucac/downloads/sheet2del.csv', sep=',')

In [152]:
Primer_Table

,Primer Name,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
0,BTX.001,ATTCTGCCGTAGGAAACTT,UF,56.1 phus,na,CAX4,na,na
1,BTX.002,cgaggcaagctaaacagatctctagacctaGTTGAACATTTCATTT...,UR,56.8 phus,na,CAX4,na,na
2,BTX.003,cagtgtcgaaaacgagctctcgagaacccttaatGATATGATCCGA...,DF,56.4 phus,na,CAX4,na,na
3,BTX.004,GCCAAACTAGTGGTACAGT,DR,57.2 phus,na,CAX4,na,na
4,BTX.005,ATCTCAGTGAATTGAAAGTTCATGC,UF,58.4 phus,na,COX12,na,na
...,...,...,...,...,...,...,...,...
57,BTX.058,gcggggacgaggcaagctaaacagatctctagacctaCTGTGCTTT...,UR,57.4 phus,na,RIM13,na,na
58,BTX.059,catccagtgtcgaaaacgagctctcgagaacccttaatTTTTGATA...,DF,56.6 phus,na,RIM13,na,na
59,BTX.060,CCCATTTTACTATTTTGGTGTTT,DR,56.4 phus,na,RIM13,na,na
60,BTX.XXX,TAGGTCTAGAGATCTGTTTAGC,F,56.3 phus,na,KanMX (pCfB 2312),na,na


In [25]:
#create a PCR table
tube_label = []
template = []
tm=[]
Fwd = []
Rev = []
Len = []
for i, gene in enumerate(genes2KO):
    
    #upstream PCR
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'UF')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'UR')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.up')
    Len.append(homology_length)
    #downstream PCR
    
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'DF')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'DR')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.dw')
    Len.append(homology_length)
    
df = np.stack([template, tube_label, tm , Len, Fwd, Rev ] ).T
PCR_Table=pd.DataFrame(df , columns=[ 'Template', 'tube_label', 'tm','Product (bp)', 'Fwd','Rev'])

PCR_Table.set_index('Template')

PCR_Table.to_csv('/Users/paucac/downloads/sheet2del2.csv', sep=',')

In [26]:
PCR_Table

,Template,tube_label,tm,Product (bp),Fwd,Rev
0,CAX4,CAX4.up,60C,400,BTX.001,BTX.002
1,CAX4,CAX4.dw,60C,400,BTX.003,BTX.004
2,COX12,COX12.up,60C,400,BTX.005,BTX.006
3,COX12,COX12.dw,60C,400,BTX.007,BTX.008
4,CUR1,CUR1.up,60C,400,BTX.009,BTX.010
5,CUR1,CUR1.dw,60C,400,BTX.011,BTX.012
6,DLD2,DLD2.up,60C,400,BTX.013,BTX.014
7,DLD2,DLD2.dw,60C,400,BTX.015,BTX.016
8,KGD2,KGD2.up,60C,400,BTX.017,BTX.018
9,KGD2,KGD2.dw,60C,400,BTX.019,BTX.020


In [35]:
 out_gap= 150
dt.make_anneal(rec.seq[0:homology_length - out_gap], 'F' , 50)

('CTGCTCAAAAATATCCCATATATAC', 50.74756078704593)

In [111]:
#design the confirmation primers

all_primers =[]
all_temp = []
position = []
Gene= []
NA =[]
Primer_name=[]
primer_n = 0

for gene in genes2KO:
    seq= part_seq[gene]
    Up = dt.make_anneal(seq, 'F' , 50)
    Dw = dt.make_anneal(seq, 'R' , 50)
    all_primers.append(Up[0])
    all_temp.append(Up[1])
    position.append('U.out')
    Gene.append(gene)
    NA.append('NA')
    Primer_name.append('BTX.XXX')
    
    all_primers.append( Dw[0])
    all_temp.append(Dw[1])
    position.append('D.out')
    Gene.append(gene)
    Primer_name.append('BTX.XXX')
    NA.append('NA')
df = np.stack([Primer_name, all_primers, position, all_temp,NA, Gene, NA, NA] ).T
Primer_Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Primer_Table.set_index('Primer Name')

Primer_Table.to_csv('/Users/paucac/downloads/sheet2del.csv', sep=',')


In [113]:
Primer_Table


,Primer Name,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
0,BTX.XXX,ATATATTTTATATATTGCCCTCAAAACT,U.out,50.95450430740635,NA,CAX4,NA,NA
1,BTX.XXX,AAACACGCTCTTGAAATTTTTAAAG,D.out,50.55591985636585,NA,CAX4,NA,NA
2,BTX.XXX,TCAGGCTGTTGATGAAGGT,U.out,51.59178690657944,NA,COX12,NA,NA
3,BTX.XXX,CAGATTTAGGGACTGAGGAA,D.out,50.286252069325144,NA,COX12,NA,NA
4,BTX.XXX,TTCTGATTGTAATATAATGACAAATTTTAATC,U.out,50.82681892845085,NA,CUR1,NA,NA
5,BTX.XXX,TTCCGTTTTATATGTTTATATTCATTGA,D.out,50.759322430542284,NA,CUR1,NA,NA
6,BTX.XXX,CAAGGTCTTCTCTACTGGC,U.out,50.75125960966665,NA,DLD2,NA,NA
7,BTX.XXX,ATGAATCAGAAGTTTTAGTTGATAGAA,D.out,50.21038959967666,NA,DLD2,NA,NA
8,BTX.XXX,TCAACTATTTAGGATTTTGTAAAGAAAA,U.out,50.42750204944821,NA,KGD2,NA,NA
9,BTX.XXX,GTGGGACATATTCGAACGT,D.out,51.19660725263975,NA,KGD2,NA,NA


## Create primers for KOing TAL1 and ZWF1 in CCM strains

In [154]:

path2parts = '/Users/paucac/Documents/PhD/CAB/CRISPA/CCM/DNA_parts/KO_confirmations/'
os.listdir(path2parts)

['zwf1.gb', 'tal1.gb']

In [155]:
## dict storing file names of parts:
genes2KO=['zwf1','tal1']
genes2KO= [g.upper() for g in genes2KO]
part_file={}
for gene in genes2KO:
    part_file[gene]= gene+".gb"

part_seq={}
part_seq['KanMX']='taggtctagagatctgtttagcttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgagctctcgagaacccttaat'

#Defines the Overhangs with KanMX  
KanMX_overhang_Up = dt.reverse_complement(part_seq['KanMX'][0:40]) #we copy the reverse complement for the upstream up
KanMX_overhang_Dw = part_seq['KanMX'][-40:]


#extract the sequence [upper homology:CDS:lower homology]

for gene in genes2KO:
    #open .gb file
    start=0
    stop=0
    recs = [rec for rec in SeqIO.parse(path2parts + part_file[gene], "genbank")]
    for rec in recs:
        #loop through feature to features to find the location  of the CDS:
        for feat in rec.features:
            if gene + ' gene'in feat.qualifiers['label']:
                location= feat.location
                start = location.start.position
                stop = location.end.position
        #stores seq in dict
        
        part_seq[gene] =  str(rec.seq[start-homology_length-out_gap:stop+homology_length+out_gap])
        


        
#design the KanMX primers
Kan_up= dt.make_anneal(part_seq['KanMX'].upper(), 'F' , 50)
Kan_dw= dt.make_anneal(part_seq['KanMX'], 'R' , 50)

#designs the primers for KO:
importlib.reload(dt)
UF_list=[]
UR_list=[]
DF_list=[]
DR_list=[]
for gene in genes2KO:
    #design the upstream F and R primers
    seq= part_seq[gene]
    UF = dt.make_anneal( seq[out_gap:out_gap+homology_length], 'F' , 50)
    UR = dt.make_anneal( seq[out_gap:out_gap+homology_length], 'R' , 50)
    
    #design the downstream F and R primers:
    DF = dt.make_anneal( seq[-homology_length-out_gap:-out_gap], 'F' , 50)
    DR = dt.make_anneal( seq[-homology_length-out_gap:-out_gap], 'R' , 50)

    #copy the KanMX 30bp overhang to the UR and DF:
    UR =  (( KanMX_overhang_Up).lower() + UR[0], UR[1])

    DF =  ((KanMX_overhang_Dw).lower() + DF[0], DF[1])
    
    UF_list.append(UF)
    UR_list.append(UR)
    DF_list.append(DF)
    DR_list.append(DR)
    
#sort the ouptut into a nice data frame

all_primers =[]
all_temp =[]
position = []
Gene= []
NA =[]
Primer_name=[]

primer_n = 77 #to match your current primer indexing
for i, gene in enumerate(genes2KO):

        
    all_primers.append(UF_list[i][0])
    all_temp.append(str(round(UF_list[i][1] ,1) +6 ) + ' phus')      #we add 6C for NEB phusion
    position.append('UF')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('CCM.'+str(i+1+3*i +primer_n ).zfill(3))

    
    all_primers.append(UR_list[i][0])
    all_temp.append(str(round(UR_list[i][1],1) + 6)+ ' phus')   
    position.append('UR')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('CCM.'+str(i+2+3*i +primer_n).zfill(3))
    
    all_primers.append(DF_list[i][0])
    all_temp.append(str(round(DF_list[i][1],1) + 6)+ ' phus')   
    position.append('DF')
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('CCM.'+str(i+3+3*i +primer_n).zfill(3))
    
    all_primers.append(DR_list[i][0])
    all_temp.append(str(round(DR_list[i][1],1) + 6)+ ' phus')   
    position.append('DR')               
    Gene.append(gene)
    NA.append('na')
    Primer_name.append('CCM.'+str(i+4+3*i +primer_n).zfill(3))
 
#add the KanMX primers
all_primers.append(Kan_up[0])
all_temp.append(str(round(Kan_up[1],1) + 6)+ ' phus')   
position.append('F')
Gene.append('KanMX (pCfB 2312)')
NA.append('na')
Primer_name.append('BTX.'+'XXX')

all_primers.append(Kan_dw[0])
all_temp.append(str(round(Kan_dw[1],1) + 6)+ ' phus')   
position.append('R')
Gene.append('KanMX (pCfB 2312)')
NA.append('na')
Primer_name.append('BTX.'+'XXX')


#some primers are over IDT 60bp limit we shorten there homology region to KanMX:
for i, primer in enumerate(all_primers):
    if len(primer) >60:
        all_primers[i] = primer[-60:]
        if  (sum([bp.islower() for bp in all_primers[i]]) < 30):
            print('WARNING your homology overhang was cut to <30bp')                    
                            
df = np.stack([Primer_name, all_primers,position, all_temp,NA, Gene, NA, NA] ).T
Primer_Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Primer_Table.set_index('Primer Name')

Primer_Table.to_csv('/Users/paucac/downloads/sheet2del.csv', sep=',')        


In [156]:
Primer_Table

,Primer Name,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
0,CCM.078,GAATAGCTAATCAAGTGGATAAGA,UF,56.1 phus,na,ZWF1,na,na
1,CCM.079,ggggacgaggcaagctaaacagatctctagacctaAAAAATGCAAG...,UR,56.7 phus,na,ZWF1,na,na
2,CCM.080,ccatccagtgtcgaaaacgagctctcgagaacccttaatCTTGCCT...,DF,56.2 phus,na,ZWF1,na,na
3,CCM.081,TAAGATGCATACTCCGGC,DR,56.8 phus,na,ZWF1,na,na
4,CCM.082,TGGCTGCAGCTCAAGA,UF,56.2 phus,na,TAL1,na,na
5,CCM.083,gggacgaggcaagctaaacagatctctagacctaGGAAGTATCTCG...,UR,56.2 phus,na,TAL1,na,na
6,CCM.084,catccagtgtcgaaaacgagctctcgagaacccttaatTATGTACA...,DF,56.8 phus,na,TAL1,na,na
7,CCM.085,TTTTGGCTGAGCTACTGG,DR,56.7 phus,na,TAL1,na,na
8,BTX.XXX,TAGGTCTAGAGATCTGTTTAGC,F,56.3 phus,na,KanMX (pCfB 2312),na,na
9,BTX.XXX,ATTAAGGGTTCTCGAGAGC,R,56.1 phus,na,KanMX (pCfB 2312),na,na


In [160]:
#design the confirmation primers

all_primers =[]
all_temp = []
position = []
Gene= []
NA =[]
Primer_name=[]
primer_n = 0

for gene in genes2KO:
    seq= part_seq[gene]
    Up = dt.make_anneal(seq, 'F' , 50)
    Dw = dt.make_anneal(seq, 'R' , 50)
    all_primers.append(Up[0])
    all_temp.append(round(Up[1],1))
    position.append('U.out')
    Gene.append(gene)
    NA.append('NA')
    Primer_name.append('CCM.XXX')
    
    all_primers.append( Dw[0])
    all_temp.append(round(Dw[1],1))
    position.append('D.out')
    Gene.append(gene)
    Primer_name.append('CCM.XXX')
    NA.append('NA')
df = np.stack([Primer_name, all_primers, position, all_temp,NA, Gene, NA, NA] ).T
Primer_Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Primer_Table.set_index('Primer Name')

Primer_Table.to_csv('/Users/paucac/downloads/sheet2del.csv', sep=',')

In [161]:
Primer_Table

,Primer Name,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
0,CCM.XXX,AAAGCATATGCATATTGCGTA,U.out,50.0,NA,ZWF1,NA,NA
1,CCM.XXX,AGCACCTGGTAAGTAAGG,D.out,50.6,NA,ZWF1,NA,NA
2,CCM.XXX,GATGCTGAAAATATAGACCTGG,U.out,50.8,NA,TAL1,NA,NA
3,CCM.XXX,CCCTGTTTTTCTGAAGCAGC,D.out,53.0,NA,TAL1,NA,NA


In [15]:
#create a PCR table
tube_label = []
template = []
tm=[]
Fwd = []
Rev = []
Len = []
for i, gene in enumerate(genes2KO):
    
    #upstream PCR
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'UF')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'UR')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.up')
    Len.append(homology_length)
    #downstream PCR
    
    template.append(gene)
    tm.append('60C')
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'DF')
    Fwd.append(Primer_Table['Primer Name'][i].values[0])
    i = (Primer_Table['Template']== gene) & (Primer_Table['Dir']== 'DR')
    Rev.append(Primer_Table['Primer Name'][i].values[0])
    tube_label.append(gene +'.dw')
    Len.append(homology_length)
    
df = np.stack([ tube_label, template,tm , Len, Fwd, Rev ] ).T
PCR_Table=pd.DataFrame(df , columns=[  'Tube_label','Template', 'tm','Product (bp)', 'Fwd','Rev'])

PCR_Table.set_index('Template')

PCR_Table.to_csv('/Users/paucac/downloads/sheet2del2.csv', sep=',')
PCR_Table

,Tube_label,Template,tm,Product (bp),Fwd,Rev
0,ZWF1.up,ZWF1,60C,400,CCM.078,CCM.079
1,ZWF1.dw,ZWF1,60C,400,CCM.080,CCM.081
2,TAL1.up,TAL1,60C,400,CCM.082,CCM.083
3,TAL1.dw,TAL1,60C,400,CCM.084,CCM.085
